In [5]:
import warnings
warnings.simplefilter(action='ignore')

import numpy as np
import pandas as pd

import pickle
from normalize import HandcraftedFeaturePreprocessor
import norm_scales

In [6]:
hier = ['Transient'] * 4 + ['Stochastic'] * 5 + ['Periodic'] * 5
cls = ['SLSN', 'SNII', 'SNIa', 'SNIbc', 'AGN', 'Blazar', 'CV/Nova', 'QSO', 'YSO', 'CEP', 'DSCT', 'E', 'RRL', 'LPV']


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, TimeDistributed, Dense, Masking, concatenate, GRU, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from math import dist
import keras
from keras import Model

def make_classifier_model(input_size, latent_size, n_cls):

    num_classes = n_cls
    n_features = 4

    input_1 = Input(input_size, name='lc')  # X.shape = (Nobjects, Ntimesteps, 4) CHANGE

    dense1 = Dense(64, activation='tanh')(input_1)

    dense2 = Dense(32, activation='relu')(dense1)

    dense3 = Dense(latent_size, activation='relu', name='latent')(dense2)

    output = Dense(n_cls, activation='softmax', name='output')(dense3)

    model = Model(inputs=[input_1], outputs=output)

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    
    return model

from sklearn.ensemble import IsolationForest

class mcif:
    def __init__(self, n_estimators = 400):
        self.n_estimators=n_estimators

    def train(self, x_data, labels):
        self.classes = np.unique(labels, axis=0)
        self.iforests = [IsolationForest(n_estimators=self.n_estimators) for i in self.classes]
        self.weights = []
        assert(len(labels) == len(x_data))
        labels=np.array(labels)
        for ind, cls in enumerate(self.classes):
            here = []
            for i in range(len(x_data)):
                if (list(cls) == list(labels[i])):
                    here.append(x_data[i])
                    
            
            self.weights.append(len(here) / len(x_data))
            self.iforests[ind].fit(here)
            

    def score_discrete(self, data):
        scores = [-det.decision_function(data) for det in self.iforests]

        scores = np.array(scores)
        scores = scores.T

        return scores

    def score(self, data):
        return [np.min(i) for i in self.score_discrete(data)]

2024-07-08 18:16:35.475780: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-08 18:16:37.298353: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 18:16:37.299980: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 18:16:37.499796: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-08 18:16:37.893397: I tensorflow/core/platform/cpu_feature_guar

In [8]:
data_pth = 'data/Deep SVDD'
# from astromcad.astromcad import *
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import RocCurveDisplay, roc_curve, roc_auc_score, auc


def auroc(in_scores, out_scores, eps=1e-5): # Copied from https://github.com/mperezcarrasco/AnomalyALeRCE
    
    scores = np.concatenate((in_scores, out_scores), axis=0)
    start = np.min(scores)
    end = np.max(scores)   
    gap = (end - start)/100000

    tprs = []
    fprs = []
    for delta in np.arange(end, start, -gap):
        tpr = np.sum(np.sum(out_scores >= delta)) / (float(len(out_scores) + eps))
        fpr = np.sum(np.sum(in_scores >= delta)) / (float(len(in_scores) + eps))
        tprs.append(tpr)
        fprs.append(fpr)
    return auc(fprs, tprs), fprs, tprs



def get_auroc(fold, hierClass, outlier, latent_size=100):
    
    feature_list_pt = data_pth + '/features_BHRF_model.pkl'

    feature_list = pd.read_pickle(feature_list_pt)

    train = pd.read_pickle('{}/train_data_filtered.pkl'.format(data_pth))
    test = pd.read_pickle('{}/test_data_filtered.pkl'.format(data_pth))

    train = train[train.classALeRCE != 'Periodic-Other']
    test = test[test.classALeRCE != 'Periodic-Other']

    
    train = train[train.hierClass==hierClass]
    test = test[test.hierClass==hierClass]
    train = train[train.classALeRCE!=outlier]

    fold_ixs = pd.read_pickle('{}/fold_{}_ixs.pkl'.format(data_pth, fold))
    val = train[(train.index.isin(fold_ixs)==False)]
    train = train[(train.index.isin(fold_ixs))]
    
    

    feature_preprocessor = HandcraftedFeaturePreprocessor()

    X_train = feature_preprocessor.preprocess(train[feature_list]).values
    X_val = feature_preprocessor.preprocess(val[feature_list]).values
    X_test = feature_preprocessor.preprocess(test[feature_list]).values
    

    test_labels = np.where((test['classALeRCE']!= outlier), 0, test['classALeRCE'])
    test_labels = np.where(test['classALeRCE']==outlier, 1, test_labels)
    test_labels = test_labels.reshape(-1,).astype('int8')


    enc = OneHotEncoder(handle_unknown='ignore')

    y_train = enc.fit_transform(np.array(train['classALeRCE']).reshape(-1, 1)).todense()
    y_val = enc.transform(np.array(val['classALeRCE']).reshape(-1, 1)).todense()
    
    single_val = [np.argmax(i) for i in y_val]
    single_train = [np.argmax(i) for i in y_train] + single_val    
    
    class_weights = {i : 0 for i in range(y_train.shape[1])}

    for value in y_train:
      class_weights[np.argmax(value)]+=1

    for id in class_weights.keys():
      class_weights[id] = len(y_train) / class_weights[id]
    
    
    model = make_classifier_model(X_train.shape[-1], latent_size, y_train.shape[-1])
    
    early_stopping = EarlyStopping(
                              patience=5,
                              min_delta=0.001,                               
                              monitor="val_loss",
                              restore_best_weights=True
                              )




    history = model.fit(x = [X_train], validation_data=([X_val], y_val), y = y_train, epochs=40, batch_size = 16, class_weight = class_weights, callbacks=[early_stopping])

    
    val_score = model.predict(X_val)
    
    latent_model = keras.Model(inputs=[model.get_layer('lc').input], outputs=[model.get_layer('latent').output])# , model.get_layer('output').output])
    
    test_latent = latent_model.predict(X_test)
#     test_latent = np.concatenate([test_latent[0], test_latent[1]], axis=1)
    train_latent = latent_model.predict(np.concatenate([X_train, X_val], axis=0))
#     train_latent = np.concatenate([train_latent[0], train_latent[1]], axis=1)
    val_latent = latent_model.predict(np.concatenate([X_val], axis=0))
#     val_latent = np.concatenate([val_latent[0], val_latent[1]], axis=1)
    
    
    det = IsolationForest()
    det.fit(train_latent)
    
    scores = np.array(-det.decision_function(test_latent))
    test_scores = scores[test_labels == 0]
    anom_scores = scores[test_labels == 1]
    sing_res = auroc(test_scores, anom_scores)[0]
    
    mcif_labels = np.concatenate([y_train, y_val], axis=0)
    det = mcif()
    det.train(train_latent, mcif_labels)

    
    scores = np.array(det.score(test_latent))
    test_scores = scores[test_labels == 0]
    anom_scores = scores[test_labels == 1]
    
    normal = auroc(test_scores, anom_scores)[0]

    
    
    
    return sing_res, normal, model.evaluate(X_val, y_val), silhouette_score(val_latent, single_val)
    
    


In [9]:
# final_mcif_res = {}
# final_iforest_res = {}

# class_accuracy = {}
# class_sil = {}
for i in range(0, 4):
    iforest_res = []
    mcif_res = []
    acc_res = []
    sil_res = []
    for fold in range(5):
        ifo, mc, _, sil = get_auroc(fold, hier[i], cls[i], latent_size=16)
        print(cls[i], fold, ifo, mc, _, sil)
        iforest_res.append(ifo)
        mcif_res.append(mc)
        acc_res.append(_)
        sil_res.append(sil)
    
    class_sil[cls[i]] = sil_res
    final_mcif_res[cls[i]] = mcif_res
    final_iforest_res[cls[i]] = iforest_res
    class_accuracy[cls[i]] = acc_res

Epoch 1/40
96/96 [==============================] - 1s 4ms/step - loss: 3.0403 - accuracy: 0.4176 - val_loss: 0.8203 - val_accuracy: 0.7374
Epoch 2/40
96/96 [==============================] - 0s 3ms/step - loss: 2.5394 - accuracy: 0.6454 - val_loss: 0.7384 - val_accuracy: 0.7714
Epoch 3/40
96/96 [==============================] - 0s 3ms/step - loss: 2.2891 - accuracy: 0.6717 - val_loss: 0.7555 - val_accuracy: 0.6800
Epoch 4/40
96/96 [==============================] - 0s 3ms/step - loss: 2.1750 - accuracy: 0.7006 - val_loss: 1.0393 - val_accuracy: 0.4361
Epoch 5/40
96/96 [==============================] - 0s 3ms/step - loss: 2.0024 - accuracy: 0.6829 - val_loss: 0.5972 - val_accuracy: 0.7995
Epoch 6/40
96/96 [==============================] - 0s 3ms/step - loss: 1.9714 - accuracy: 0.7354 - val_loss: 0.7310 - val_accuracy: 0.7057
Epoch 7/40
96/96 [==============================] - 0s 3ms/step - loss: 1.7684 - accuracy: 0.7689 - val_loss: 0.6506 - val_accuracy: 0.7351
Epoch 8/40
96/96 [==

Epoch 1/40
99/99 [==============================] - 1s 4ms/step - loss: 3.1052 - accuracy: 0.3990 - val_loss: 0.9953 - val_accuracy: 0.3142
Epoch 2/40
99/99 [==============================] - 0s 3ms/step - loss: 2.6219 - accuracy: 0.4765 - val_loss: 0.9875 - val_accuracy: 0.3017
Epoch 3/40
99/99 [==============================] - 0s 3ms/step - loss: 2.2908 - accuracy: 0.6550 - val_loss: 0.7338 - val_accuracy: 0.7269
Epoch 4/40
99/99 [==============================] - 0s 3ms/step - loss: 2.1591 - accuracy: 0.7141 - val_loss: 0.6927 - val_accuracy: 0.7394
Epoch 5/40
99/99 [==============================] - 0s 3ms/step - loss: 2.0510 - accuracy: 0.7236 - val_loss: 0.7677 - val_accuracy: 0.6559
Epoch 6/40
99/99 [==============================] - 0s 3ms/step - loss: 1.9297 - accuracy: 0.7687 - val_loss: 0.9152 - val_accuracy: 0.4763
Epoch 7/40
99/99 [==============================] - 0s 3ms/step - loss: 1.8711 - accuracy: 0.7141 - val_loss: 0.6355 - val_accuracy: 0.7893
Epoch 8/40
99/99 [==

Epoch 4/40
79/79 [==============================] - 0s 3ms/step - loss: 2.0072 - accuracy: 0.6419 - val_loss: 0.4348 - val_accuracy: 0.8848
Epoch 5/40
79/79 [==============================] - 0s 3ms/step - loss: 1.8036 - accuracy: 0.7161 - val_loss: 0.7704 - val_accuracy: 0.5591
Epoch 6/40
79/79 [==============================] - 0s 3ms/step - loss: 1.6997 - accuracy: 0.7169 - val_loss: 0.9861 - val_accuracy: 0.4409
Epoch 7/40
79/79 [==============================] - 0s 3ms/step - loss: 1.5573 - accuracy: 0.7225 - val_loss: 0.4944 - val_accuracy: 0.8126
Epoch 8/40
79/79 [==============================] - 0s 3ms/step - loss: 1.4187 - accuracy: 0.7089 - val_loss: 0.5339 - val_accuracy: 0.7819
Epoch 9/40
21/21 [==============================] - 0s 1ms/step - loss: 0.4348 - accuracy: 0.8848
SNII 3 0.8241930353888243 0.6883620590483903 [0.4347519278526306, 0.8847926259040833] 0.18474233
Epoch 1/40
77/77 [==============================] - 1s 4ms/step - loss: 3.1262 - accuracy: 0.1514 - val_l

Epoch 8/40
29/29 [==============================] - 0s 3ms/step - loss: 1.7337 - accuracy: 0.6958 - val_loss: 0.8059 - val_accuracy: 0.6388
Epoch 9/40
29/29 [==============================] - 0s 3ms/step - loss: 1.5739 - accuracy: 0.7199 - val_loss: 0.7784 - val_accuracy: 0.6426
Epoch 10/40
29/29 [==============================] - 0s 3ms/step - loss: 1.4143 - accuracy: 0.7221 - val_loss: 0.6506 - val_accuracy: 0.7300
Epoch 11/40
29/29 [==============================] - 0s 3ms/step - loss: 1.4636 - accuracy: 0.7309 - val_loss: 0.7547 - val_accuracy: 0.6844
Epoch 12/40
29/29 [==============================] - 0s 3ms/step - loss: 1.1885 - accuracy: 0.7549 - val_loss: 0.6506 - val_accuracy: 0.7529
Epoch 13/40
29/29 [==============================] - 0s 3ms/step - loss: 1.1069 - accuracy: 0.8053 - val_loss: 0.6510 - val_accuracy: 0.7643
Epoch 14/40
29/29 [==============================] - 0s 3ms/step - loss: 0.9931 - accuracy: 0.8118 - val_loss: 0.6427 - val_accuracy: 0.7567
Epoch 15/40
29/

Epoch 11/40
8/8 [==============================] - 0s 1ms/step - loss: 0.7163 - accuracy: 0.7056
SNIa 4 0.6513868895539211 0.5621462132278182 [0.7162737250328064, 0.7056451439857483] 0.018290272
Epoch 1/40
93/93 [==============================] - 1s 4ms/step - loss: 3.2025 - accuracy: 0.6732 - val_loss: 0.9745 - val_accuracy: 0.6090
Epoch 2/40
93/93 [==============================] - 0s 3ms/step - loss: 2.6785 - accuracy: 0.7131 - val_loss: 0.5782 - val_accuracy: 0.7832
Epoch 3/40
93/93 [==============================] - 0s 3ms/step - loss: 2.3042 - accuracy: 0.7700 - val_loss: 0.5676 - val_accuracy: 0.7867
Epoch 4/40
93/93 [==============================] - 0s 3ms/step - loss: 2.0677 - accuracy: 0.8112 - val_loss: 0.5670 - val_accuracy: 0.7879
Epoch 5/40
93/93 [==============================] - 0s 3ms/step - loss: 1.8222 - accuracy: 0.7963 - val_loss: 0.4256 - val_accuracy: 0.8341
Epoch 6/40
93/93 [==============================] - 0s 3ms/step - loss: 1.6192 - accuracy: 0.8281 - val_l

27/27 [==============================] - 0s 1ms/step - loss: 0.3948 - accuracy: 0.8648
SNIbc 2 0.606903661030281 0.6266901296319486 [0.3947884142398834, 0.8648325204849243] 0.3467114
Epoch 1/40
96/96 [==============================] - 1s 4ms/step - loss: 3.1578 - accuracy: 0.3359 - val_loss: 0.8552 - val_accuracy: 0.6742
Epoch 2/40
96/96 [==============================] - 0s 3ms/step - loss: 2.5475 - accuracy: 0.7065 - val_loss: 0.6079 - val_accuracy: 0.7715
Epoch 3/40
96/96 [==============================] - 0s 3ms/step - loss: 2.0042 - accuracy: 0.7693 - val_loss: 0.5908 - val_accuracy: 0.7588
Epoch 4/40
96/96 [==============================] - 0s 3ms/step - loss: 1.6591 - accuracy: 0.8065 - val_loss: 0.5393 - val_accuracy: 0.7854
Epoch 5/40
96/96 [==============================] - 0s 3ms/step - loss: 1.4773 - accuracy: 0.8301 - val_loss: 0.5041 - val_accuracy: 0.8144
Epoch 6/40
96/96 [==============================] - 0s 3ms/step - loss: 1.3653 - accuracy: 0.8477 - val_loss: 0.6970 

In [12]:
for i in range(4):
    print(cls[i], np.mean(final_mcif_res[cls[i]]), np.mean(final_iforest_res[cls[i]]))

SLSN 0.49985738461439383 0.7153235333899722
SNII 0.6804656489498222 0.8036191791006319
SNIa 0.549669329972555 0.6324053836898338
SNIbc 0.5645620417096625 0.5496937205194168


In [7]:
for i in range(len(cls)):
    print(cls[i], np.median(final_mcif_res[cls[i]]), np.median(final_iforest_res[cls[i]]))

KeyError: 'SLSN'

In [8]:
for i in range(len(cls)):
    print(cls[i], np.mean(final_mcif_res[cls[i]]), np.mean(final_iforest_res[cls[i]]))

KeyError: 'SLSN'

In [17]:
for i in range(4):
    print(cls[i], np.median(final_mcif_res[cls[i]]), np.median(final_iforest_res[cls[i]]))

SLSN 0.6441545773656486 0.7579898807573691
SNII 0.7330336095000107 0.8066324371412134
SNIa 0.5312972384601451 0.5817525439214417
SNIbc 0.5612097299429771 0.5531670286768318


In [6]:
print(final_mcif_res)
print(final_iforest_res)

{'AGN': [0.5942888431293877, 0.604263135909873, 0.5526579186560022, 0.6237640789291634, 0.5044910432604891], 'Blazar': [0.7528678878794575, 0.7398070209107709, 0.6965472412111107, 0.7067220704276689, 0.5377922792430524], 'CV/Nova': [0.9110099995935691, 0.9315064816942131, 0.9082980539028332, 0.9439195929897694, 0.9451370141394422], 'QSO': [0.4848702707513644, 0.5621191882831116, 0.4924275807140116, 0.6824670618812855, 0.36944349931104653], 'YSO': [0.7564813925669007, 0.8520701119375986, 0.9084617954778097, 0.9157296134979201, 0.8721345490801498], 'CEP': [0.818814764210829, 0.8357457139757536, 0.8563672339582423, 0.8476828406698085, 0.8761514112831182], 'DSCT': [0.797047345493604, 0.8099035958367641, 0.8344295187144936, 0.6811581135945859, 0.8262551392419709], 'E': [0.8761374897023679, 0.7658830471562067, 0.8227519970357268, 0.8349754906069093, 0.6859799200347856], 'RRL': [0.8310239112979566, 0.737519370257954, 0.6709443043375606, 0.8137798330360881, 0.7821138837950711], 'LPV': [0.83612

In [11]:

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

In [12]:
save_object(final_mcif_res, 'final_mcif_res')

In [13]:
save_object(final_iforest_res, 'final_iforest_res')

In [14]:
save_object(class_accuracy, 'class_accuracy')

In [15]:
save_object(class_sil, 'class_sil')

In [1]:

def load_object(filename):
    with open(filename, 'rb') as inp:
        obj = pickle.load(inp)
    return obj

In [4]:
final_mcif_res = load_object('final_mcif_res')
final_iforest_res = load_object('final_iforest_res')
class_accuracy = load_object('class_accuracy')
class_sil = load_object('class_sil')

In [20]:
for i in range(0, len(cls)):
#     for j in range(5):
#         print(round(final_mcif_res[cls[i]][j], 2), round(final_iforest_res[cls[i]][j], 2), round(class_sil[cls[i]][j], 2))
#     print('\n')
    print(cls[i], round(np.median(final_mcif_res[cls[i]]), 2), round(np.median(final_iforest_res[cls[i]]), 2)) # round(np.median(np.array(class_accuracy[cls[i]])[:, 1]), 2))
#     print(cls[i], np.median(final_mcif_res[cls[i]]) > np.median(final_iforest_res[cls[i]]), round(np.mean(class_sil[cls[i]]), 2)) # round(np.median(np.array(class_accuracy[cls[i]])[:, 1]), 2))

SLSN 0.63 0.76
SNII 0.71 0.82
SNIa 0.56 0.64
SNIbc 0.56 0.54
AGN 0.59 0.7
Blazar 0.71 0.71
CV/Nova 0.93 0.94
QSO 0.49 0.37
YSO 0.87 0.97
CEP 0.85 0.77
DSCT 0.81 0.61
E 0.82 0.86
RRL 0.78 0.88
LPV 0.78 0.97
